In [362]:
import pymysql.cursors

# Connect to the database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='',
                             db='printerproject',
                             cursorclass=pymysql.cursors.DictCursor)

try:
    with connection.cursor() as cursor:
        # Read a single record
        sql = "SELECT `id_Etudiant`, `Livre_name`, `Livre_theme` FROM `history`, `books` WHERE books.ISBN = history.ISBN"
        cursor.execute(sql)
        result = cursor. fetchall()
        #print(result)
        sql = "SELECT `Id_Etudiant`, `Livre_theme`, COUNT(*) as C FROM `history`, `books` WHERE books.ISBN = history.ISBN GROUP BY Livre_theme, Id_Etudiant"
        cursor.execute(sql)
        result2 = cursor. fetchall()
        #print(result2)
        sql = "SELECT `Id_Etudiant`, `Livre_theme` FROM `history`, `books` WHERE books.ISBN = history.ISBN GROUP BY Livre_theme, Id_Etudiant"
        cursor.execute(sql)
        result3 = cursor. fetchall()
        #print(result3)
        sql = "SELECT MAX(`Id_Etudiant`)FROM `history`"
        cursor.execute(sql)
        max_id = cursor. fetchall()
        print(max_id)
finally:
    connection.close()
#print(result3)
themes_per_id = [];
max_id[0]['MAX(`Id_Etudiant`)'] = max_id[0]['MAX(`Id_Etudiant`)'] + 1
max_id_int = max_id[0]['MAX(`Id_Etudiant`)']
print(max_id_int)
print(len(result3))
#for i in range(0, len(result3)):
#    s = result3[i]['Livre_theme']
#    result3[i]['Livre_theme']=[]
#    result3[i]['Livre_theme'].append(s)
for i in range(0, max_id_int):
    l = len(result3)
    for j in range(0, len(result3)):
        if(result3[j]['Id_Etudiant'] == result3[i]['Id_Etudiant'] and j != i):
            #result3[i]['Livre_theme'].append(result3[j]['Livre_theme'])
            result3[i]['Livre_theme']+=","
            result3[i]['Livre_theme']+=(result3[j]['Livre_theme'])
            result3[j]['Livre_theme'] = ""
result4 = []
for j in range(0, len(result3)):
    if(result3[j]['Livre_theme'] != ""):
        result3[j]['Livre_theme'] = [x for x in result3[j]['Livre_theme'].split(',') if x]
        result4.append(result3[j])
#print(result4)
#print(result4[0]['Livre_theme'])
#print(result4[73]['Livre_theme'])
def get_max_length(array1, array2):
    if(len(array1) > len(array2)):
        return 1
    return 2
def count_different(array1, array2):
    ans = 0
    s1 = set(array1)
    s2 = set(array2)
    temp1 = [x for x in array1 if x not in s2]
    temp2 = [x for x in array2 if x not in s1]
    temp3 = temp1 + temp2
    return temp3
#print(count_different(result4[0]['Livre_theme'],result4[73]['Livre_theme']))
from operator import itemgetter
newlist = sorted(result4, key=itemgetter('Id_Etudiant')) 
#print(newlist)

[{'MAX(`Id_Etudiant`)': 100}]
101
196


In [363]:
import numpy as np
dist_matrix = np.zeros((max_id_int, max_id_int))
dist_matrix[:] = np.nan
print(max_id_int)
def get_index(result, index):
    ans = -1;
    for i in range(len(result)):
        if result[i]['Id_Etudiant'] == index:
            ans = i
            return ans
    return ans
        
def distance_theme(result):
    for i in range(1, max_id_int):
        index_i_in_result = get_index(result, i)
        if index_i_in_result != -1:
            for j in range(1, max_id_int):
                index_j_in_result = get_index(result, j) 
                if index_j_in_result != -1:
                    dist_matrix[i][j] = len(count_different(result[index_i_in_result]['Livre_theme'], result[index_j_in_result]['Livre_theme']))
                    #print("i=%d,j=%d, l=%d" % (i, j, dist_matrix[i][j]))
distance_theme(newlist)
array_max = np.nanmax(dist_matrix, axis=1)
array_max = np.nan_to_num(array_max)
array_max = set(array_max)
array_max = sorted(array_max, reverse=True)
print(array_max)

101
[13.0, 12.0, 11.0, 10.0, 9.0, 8.0, 7.0, 0.0]


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: All-NaN slice encountered


In [364]:
import math
k = 3 # number of clusters
k_copy = k
clusters_centers = []
clusters_dic = {}

# Randomly choose first two farthest Ids
def choose_first_two_centers(max_id_int, array_max):
    for i in range(1,max_id_int):
        for j in range(1, max_id_int):
            if dist_matrix[i][j] == array_max[0]:
                clusters_centers.append(i)
                clusters_centers.append(j)
                return
choose_first_two_centers(max_id_int, array_max)
k -= 2
dist_to_set = np.zeros((max_id_int, len(clusters_centers)))
dist_to_set[:] = np.nan
#choose remaining centers by calculating the distance to the set of the first two Ids

def choose_farthest_next_centers(result, max_id_int, k, dist_to_set):
    while k > 0:
        for i in range(1,max_id_int):
            index_i_in_result = get_index(result, i)
            if index_i_in_result != -1:
                for j in range(0, len(clusters_centers)):
                    if i not in clusters_centers:
                        dist_to_set[i][j] = len(count_different(result[index_i_in_result]['Livre_theme'], result[get_index(result, clusters_centers[j])]['Livre_theme']))
        array_dist_to_set = np.nanmin(dist_to_set, axis=1)
        array_dist_to_set = np.nan_to_num(array_dist_to_set)
        next_center = np.argmax(array_dist_to_set)
        clusters_centers.append(next_center)
        k -= 1
        dist_to_set = np.zeros((max_id_int, len(clusters_centers)))
        dist_to_set[:] = np.nan

choose_farthest_next_centers(newlist, max_id_int, k, dist_to_set)
    
for i in range(0,k_copy):
    key = clusters_centers[i]
    clusters_dic[key] = []

def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx

for i in range(1, max_id_int):
    values = []
    for j in clusters_centers:
        if not math.isnan(dist_matrix[i][j]):
            values.append(dist_matrix[i][j])
        else:
            values.append(math.inf)
    values = np.asarray(values)
    key = find_nearest(values, 0)
    clusters_dic[clusters_centers[key]].append(i)
print(clusters_centers)
print(clusters_dic)

[6, 38, 49]
{6: [1, 3, 4, 6, 7, 8, 9, 11, 13, 16, 17, 19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 39, 40, 41, 42, 43, 44, 46, 47, 50, 52, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 66, 67, 68, 70, 71, 72, 74, 75, 76, 77, 78, 79, 80, 81, 83, 85, 86, 89, 91, 93, 96, 97, 98, 100], 38: [5, 10, 12, 14, 18, 26, 38, 45, 48, 53, 73, 92, 94, 95], 49: [2, 15, 37, 49, 51, 57, 65, 69, 82, 84, 87, 88, 90, 99]}


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: All-NaN axis encountered


In [366]:
import random

def recommend(clusters_dic, clusters_centers, id_etudiant):
    for i in clusters_centers:
        if id_etudiant in clusters_dic[i]:
            near_id = random.choice(clusters_dic[i])
            while near_id == id_etudiant:
                near_id = random.choice(clusters_dic[i])
            return near_id
    id_generated = id_etudiant
    while id_generated == id_etudiant:
        id_generated = random.randint(1,max_id_int-2)
    return id_generated
id_recommended = recommend(clusters_dic, clusters_centers, 2)
recommended_theme = ""
for i in range(0, len(newlist)):
    if(newlist[i]['Id_Etudiant'] == id_recommended):
        recommended_theme = random.choice(newlist[i]['Livre_theme'])
        break
# Connect to the database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='',
                             db='printerproject',
                             cursorclass=pymysql.cursors.DictCursor)

try:
    with connection.cursor() as cursor:
        # Read a single record
        sql = "SELECT `Livre_name` FROM `books` WHERE books.Livre_theme = '" + recommended_theme + "'"
        cursor.execute(sql)
        books = cursor. fetchall()
finally:
    connection.close()
if(books):
    random_recommended_book = random.choice(books)['Livre_name']
else:
    while(not books):
        random_theme = random.choice(newlist)['Livre_theme'][0]
        # Connect to the database
        connection = pymysql.connect(host='localhost',
                                     user='root',
                                     password='',
                                     db='printerproject',
                                     cursorclass=pymysql.cursors.DictCursor)

        try:
            with connection.cursor() as cursor:
                # Read a single record
                sql = "SELECT `Livre_name` FROM `books` WHERE books.Livre_theme = '" + random_theme + "'"
                cursor.execute(sql)
                books = cursor. fetchall()
        finally:
            connection.close()
    random_recommended_book = random.choice(books)['Livre_name']
print(random_recommended_book)

eu tellus


In [2]:
from PIL import Image
img = Image.open('6.png').convert('LA')
img.save('6Gray.png')
#https://raspberrypi.stackexchange.com/questions/45570/how-do-i-make-serial-work-on-the-raspberry-pi3
#De not change "root=/dev/mmcblk0p2"